In [12]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm  # For progress tracking

import nltk
nltk.data.path.append("/usr/local/share/nltk_data")

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')  # In case it's needed
nltk.download('wordnet')  # If lemmatization is used
nltk.download('omw-1.4')  # Sometimes needed for WordNet
nltk.download('averaged_perceptron_tagger')  # If POS tagging is used

# Load company dataset
company_df = pd.read_csv("ml_insurance_challenge.csv")  # Adjust file path as needed

# Load insurance taxonomy labels
insurance_labels_df = pd.read_excel("insurance_taxonomy.xlsx", header=None)
insurance_labels = insurance_labels_df[0].tolist()  # Extract labels as a list


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alessiacolumban/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alessiacolumban/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alessiacolumban/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/alessiacolumban/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alessiacolumban/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [13]:
def load_glove_embeddings(file_path):
    glove_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Loading GloVe embeddings"):
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)
            glove_dict[word] = vector
    return glove_dict

# Load GloVe embeddings (change the path if needed)
glove_path = "glove.6B.300d.txt"  # Ensure this file exists
glove_embeddings = load_glove_embeddings(glove_path)

print("GloVe embeddings loaded. Example vector for 'insurance':", glove_embeddings.get("insurance", "Not found"))


Loading GloVe embeddings: 400000it [00:20, 19220.22it/s]


GloVe embeddings loaded. Example vector for 'insurance': [-5.6131e-02  1.3407e-01 -2.0990e-01  6.6576e-01  3.9511e-01  1.1705e-01
  5.9482e-01 -4.4346e-01 -6.3459e-01 -1.4874e+00  1.4473e-01 -4.0983e-02
  2.9000e-01 -6.2229e-01  7.6048e-02  2.9911e-02  4.4002e-01 -1.4550e-01
  1.4908e-01 -2.0335e-01  6.1897e-01  4.3210e-02 -6.5998e-02  4.8003e-01
  1.2430e-01  3.2287e-01 -2.0818e-01 -3.4093e-02 -4.3413e-01 -1.1550e-01
 -3.7614e-01  4.4211e-01  2.9823e-01 -4.9800e-01 -1.0069e+00 -1.5966e-01
  3.6354e-01 -3.1454e-01  6.5463e-01 -3.5102e-02 -1.1068e-01 -5.7771e-01
  5.0527e-01  4.2670e-01  1.2369e-01 -6.6690e-01 -4.9843e-01  4.4435e-01
 -2.0306e-01 -1.2199e-01 -1.9348e-02  4.6638e-01 -1.4160e-01  4.3297e-01
  7.8291e-02 -1.7717e-01  5.8133e-02  4.6367e-01  1.3563e-01 -4.2853e-01
  5.4427e-01 -7.3813e-02 -1.5729e-01 -3.8912e-02  7.5924e-01  3.2579e-01
  2.0219e-01 -4.3475e-01 -1.8684e-01 -5.8086e-01 -2.8047e-01 -2.3834e-02
  6.5123e-01  3.4918e-01  2.6611e-01  3.0088e-01 -9.2837e-01 -2.894

In [14]:
def text_to_glove_vector(text, glove_embeddings):
    words = text.split()  # Simple tokenization
    vectors = [glove_embeddings.get(word, np.zeros(300)) for word in words]  # Default to zero vector if word is unknown
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)


In [15]:
# # Step 3: Preprocess Text Data
# def preprocess_text(text):
#     if not isinstance(text, str) or pd.isna(text):  # Ensure text is valid
#         return []
#     text = text.lower()
#     text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove special characters
#     tokens = word_tokenize(text)
#     tokens = [word for word in tokens if word not in stopwords.words('english')]
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(word) for word in tokens]
#     return tokens


# # Test preprocessing
# print(preprocess_text("This is a test sentence for insurance classification!"))


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/alessiacolumban/nltk_data'
    - '/Users/alessiacolumban/opt/anaconda3/envs/Yolo_and_tensorflowmetal/nltk_data'
    - '/Users/alessiacolumban/opt/anaconda3/envs/Yolo_and_tensorflowmetal/share/nltk_data'
    - '/Users/alessiacolumban/opt/anaconda3/envs/Yolo_and_tensorflowmetal/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/local/share/nltk_data'
**********************************************************************


In [16]:
def preprocess_text(text):
    if not isinstance(text, str) or pd.isna(text):
        return []
    
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove special characters
    tokens = text.split()  # Simple alternative to word_tokenize()
    
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens


In [17]:
def text_to_glove_vector(text, embeddings, dim=300):  # Ensure 300D for GloVe 6B.300d
    if pd.isna(text) or not isinstance(text, str):
        return np.zeros(dim)  # Return a zero vector if text is missing
    
    tokens = text.split()  # Using simple tokenization
    vectors = [embeddings[word] for word in tokens if word in embeddings]
    
    if len(vectors) == 0:
        return np.zeros(dim)  # Return a zero vector if no words are found
    
    return np.mean(vectors, axis=0)  # Take the average of word vectors

# Convert company descriptions to vectors
company_df['glove_vector'] = company_df['description'].apply(lambda x: text_to_glove_vector(x, glove_embeddings))

# Convert insurance taxonomy labels to vectors
insurance_label_vectors = np.vstack([text_to_glove_vector(label, glove_embeddings) for label in insurance_labels])


In [18]:
from sklearn.metrics.pairwise import cosine_similarity

# Convert list of vectors into a NumPy array
company_vectors = np.vstack(company_df['glove_vector'].values)

# Compute similarity matrix
similarity_matrix = cosine_similarity(company_vectors, insurance_label_vectors)

# Print similarity shape (should be companies × labels)
print(f"Similarity Matrix Shape: {similarity_matrix.shape}")


Similarity Matrix Shape: (9494, 221)


In [19]:
def assign_labels(similarity_matrix, labels, top_n=3):
    """
    Assigns the top N most similar labels to each company based on cosine similarity.
    """
    assigned_labels = []
    
    for row in similarity_matrix:
        top_indices = row.argsort()[-top_n:][::-1]  # Get indices of top N labels
        best_labels = [labels[i] for i in top_indices]
        assigned_labels.append(", ".join(best_labels))  # Join multiple labels into a string
    
    return assigned_labels

# Assign labels to companies
company_df['insurance_label'] = assign_labels(similarity_matrix, insurance_labels, top_n=3)

# Display the first few results
print(company_df[['description', 'insurance_label']].head())


                                         description  \
0  Welchcivils is a civil engineering and constru...   
1  Kyoto Vegetable Specialists Uekamo, also known...   
2  Loidholdhof Integrative Hofgemeinschaft is a c...   
3  PATAGONIA Chapa Y Pintura is an auto body shop...   
4  Stanica WODNA PTTK Swornegacie is a cultural e...   

                                     insurance_label  
0  Fishing and Hunting Services, Windows and Door...  
1  Fishing and Hunting Services, Windows and Door...  
2  Fishing and Hunting Services, Windows and Door...  
3  Fishing and Hunting Services, Windows and Door...  
4  Fishing and Hunting Services, Windows and Door...  


In [20]:
company_df.to_csv("classified_companies.csv", index=False)
print("Classification results saved to 'classified_companies.csv'")


Classification results saved to 'classified_companies.csv'
